In [ ]:
import cv2
import sys
import mediapipe as mp
import math
import pyttsx3
import time
from PyQt6.QtCore import QThread, Qt, pyqtSignal, pyqtSlot
from PyQt6.QtGui import QImage, QPixmap, QFont
from PyQt6.QtWidgets import (QApplication, QMainWindow, QLabel, 
                            QLineEdit, QVBoxLayout, QWidget, 
                            QPushButton, QMessageBox)

# 初始化MediaPipe和pyttsx3
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils
engine = pyttsx3.init()

In [ ]:
def calculate_angle(a, b, c):
    # a, b, c 为三个点的坐标
    ab = [b[0] - a[0], b[1] - a[1]]  # 向量 AB
    bc = [c[0] - b[0], c[1] - b[1]]  # 向量 BC
    
    # 计算向量的点积
    dot_product = ab[0] * bc[0] + ab[1] * bc[1]
    
    # 计算向量的叉积
    cross_product = ab[0] * bc[1] - ab[1] * bc[0]
    
    # 计算夹角
    angle = math.degrees(math.atan2(abs(cross_product), dot_product))
    return angle

def speak(message):
    engine.say(message)
    engine.runAndWait()

# 初始化摄像头
cap = cv2.VideoCapture(0)

# 用来控制是否已抬腿
leg_raised = False

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # 转换为RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 获取人体关键点
    results = pose.process(frame_rgb)
    
    if results.pose_landmarks:
        # 提取关节坐标
        landmarks = results.pose_landmarks.landmark
        
        # 获取左肩、左髋、左膝、左踝、左脚大拇指的坐标
        left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y)
        left_hip = (landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y)
        left_knee = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y)
        left_ankle = (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y)
        left_big_toe = (landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y)

        # 计算髋部、膝部、踝部的角度
        hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)  # 髋关节角度
        knee_angle = calculate_angle(left_hip, left_knee, left_ankle)  # 膝关节角度
        ankle_angle = calculate_angle(left_knee, left_ankle, left_big_toe)  # 踝关节角度

        # 计算补角
        hip_angle_complement = 180 - hip_angle  # 髋关节补角
        knee_angle_complement = 180 - knee_angle  # 膝关节补角
        ankle_angle_complement = 180 - ankle_angle  # 踝关节补角

        # 在图像上显示角度数字
        # 计算适当的位置来标示角度数值
        cv2.putText(frame, f"{hip_angle_complement:.2f}°", 
                    (int(left_hip[0] * frame.shape[1]) + 10, int(left_hip[1] * frame.shape[0]) - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"{knee_angle_complement:.2f}°", 
                    (int(left_knee[0] * frame.shape[1]) + 10, int(left_knee[1] * frame.shape[0]) - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"{ankle_angle_complement:.2f}°", 
                    (int(left_ankle[0] * frame.shape[1]) + 10, int(left_ankle[1] * frame.shape[0]) - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # 计算左肩、左髋、左膝、左踝之间的连线
        cv2.line(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 
                 (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接肩膀和髋
        cv2.line(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 
                 (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接髋和膝
        cv2.line(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 
                 (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接膝和踝
        cv2.line(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 
                 (int(left_big_toe[0] * frame.shape[1]), int(left_big_toe[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接踝和大拇指

        # 绘制圆圈表示关键点
        cv2.circle(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 5, (0, 0, 255), -1)  # 左肩
        cv2.circle(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 5, (0, 255, 255), -1)  # 左髋
        cv2.circle(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 5, (255, 0, 0), -1)  # 左膝
        cv2.circle(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 5, (255, 255, 0), -1)  # 左踝
        cv2.circle(frame, (int(left_big_toe[0] * frame.shape[1]), int(left_big_toe[1] * frame.shape[0])), 5, (255, 0, 255), -1)  # 左脚大拇指

        # 检查角度范围并语音播报
        if (100 < ankle_angle_complement < 110) and (173 < knee_angle_complement < 180) and (175 < hip_angle_complement < 180):
            speak("请抬腿")
            leg_raised = True

        # 检测用户是否抬腿并检查膝关节
        if leg_raised:
            # 如果髋关节补角小于150度
            if hip_angle_complement < 150:
                if knee_angle_complement < 170:
                    speak("膝关节未伸直")
                elif hip_angle > 55:
                    if hip_angle < 70:
                        speak("正常")
                    if hip_angle > 70:
                        speak("优秀")

    # 显示结果
    cv2.imshow("Pose Detection", frame)

    # 按'q'键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import sys
import cv2
import math
import time
import mediapipe as mp
import pyttsx3
from PyQt6.QtCore import QThread, Qt, pyqtSignal, pyqtSlot
from PyQt6.QtGui import QImage, QPixmap, QFont
from PyQt6.QtWidgets import (QApplication, QMainWindow, QLabel, 
                            QLineEdit, QVBoxLayout, QWidget, 
                            QPushButton, QMessageBox,QHBoxLayout)

# 初始化MediaPipe姿势识别模型
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 初始化语音引擎
engine = pyttsx3.init()

def calculate_angle(a, b, c):
    """计算三个关节点的角度"""
    ab = [b[0] - a[0], b[1] - a[1]]
    bc = [c[0] - b[0], c[1] - b[1]]
    
    dot_product = ab[0] * bc[0] + ab[1] * bc[1]
    cross_product = ab[0] * bc[1] - ab[1] * bc[0]
    
    angle = math.degrees(math.atan2(abs(cross_product), dot_product))
    return angle

class VideoThread(QThread):
    change_pixmap = pyqtSignal(QImage)
    save_signal = pyqtSignal()
    speak_signal = pyqtSignal(str)
    error_signal = pyqtSignal(str)
    capture_signal = pyqtSignal()  

    def __init__(self):
        super().__init__()
        self.subject_id = ""
        self.stable_start = None
        self.target_angle = None
        self.current_frame = None
        self.running = True
        self.leg_raised = False
        self.capture_signal.connect(self.manual_capture)

    @pyqtSlot()
    def manual_capture(self):
        """手动捕获当前帧"""
        if self.current_frame is not None:
            self.save_signal.emit()

    def run(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            self.error_signal.emit("无法打开摄像头")
            return

        while self.running and cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                continue

            annotated_frame = frame.copy()

            # 姿势检测处理
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            self.current_frame = frame.copy()

            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark

                    # 获取关节坐标（左半身）
                    left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y)
                    left_hip = (landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                               landmarks[mp_pose.PoseLandmark.LEFT_HIP].y)
                    left_knee = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y)
                    left_ankle = (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                                 landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y)
                    left_big_toe = (landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x,
                                   landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y)

                    # 计算关节角度（补角）
                    hip_angle = 180 - calculate_angle(left_shoulder, left_hip, left_knee)
                    knee_angle = 180 - calculate_angle(left_hip, left_knee, left_ankle)
                    ankle_angle = 180 - calculate_angle(left_knee, left_ankle, left_big_toe)

                    # 绘制姿势标记
                    # 在BGR帧上绘制，后续需要转换为RGB
                    cv2.putText(frame, f"{hip_angle:.1f} degree", 
                              (int(left_hip[0] * frame.shape[1]) + 10, 
                               int(left_hip[1] * frame.shape[0]) - 10), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    
                    # 关节连线（代码保持原有逻辑）
                    cv2.putText(frame, f"{hip_angle:.2f} degree", 
                                (int(left_hip[0] * frame.shape[1]) + 10, int(left_hip[1] * frame.shape[0]) - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(frame, f"{knee_angle:.2f} degree", 
                                (int(left_knee[0] * frame.shape[1]) + 10, int(left_knee[1] * frame.shape[0]) - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(frame, f"{ankle_angle:.2f} degree", 
                                (int(left_ankle[0] * frame.shape[1]) + 10, int(left_ankle[1] * frame.shape[0]) - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # 计算左肩、左髋、左膝、左踝之间的连线
                    cv2.line(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 
                            (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接肩膀和髋
                    cv2.line(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 
                            (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接髋和膝
                    cv2.line(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 
                            (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接膝和踝
                    cv2.line(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 
                            (int(left_big_toe[0] * frame.shape[1]), int(left_big_toe[1] * frame.shape[0])), (0, 255, 0), 2)  # 连接踝和大拇指

                    # 绘制圆圈表示关键点
                    cv2.circle(frame, (int(left_shoulder[0] * frame.shape[1]), int(left_shoulder[1] * frame.shape[0])), 5, (0, 0, 255), -1)  # 左肩
                    cv2.circle(frame, (int(left_hip[0] * frame.shape[1]), int(left_hip[1] * frame.shape[0])), 5, (0, 255, 255), -1)  # 左髋
                    cv2.circle(frame, (int(left_knee[0] * frame.shape[1]), int(left_knee[1] * frame.shape[0])), 5, (255, 0, 0), -1)  # 左膝
                    cv2.circle(frame, (int(left_ankle[0] * frame.shape[1]), int(left_ankle[1] * frame.shape[0])), 5, (255, 255, 0), -1)  # 左踝
                    cv2.circle(frame, (int(left_big_toe[0] * frame.shape[1]), int(left_big_toe[1] * frame.shape[0])), 5, (255, 0, 255), -1)  # 左脚大拇指

                    annotated_frame = frame.copy()
                    cv2.putText(annotated_frame, f"kuan: {hip_angle:.2f} degree", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(annotated_frame, f"xi: {knee_angle:.2f} degree", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(annotated_frame, f"huai: {ankle_angle:.2f} degree", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    # 姿势条件判断
                    if (100 < ankle_angle < 110) and (173 < knee_angle < 180) and (175 < hip_angle < 180):
                        self.speak_signal.emit("请抬腿")
                        self.leg_raised = True



                    # if leg_raised:
                    #     # 如果髋关节补角小于150度
                    #     if hip_angle_complement < 150:
                    #         if knee_angle_complement < 170:
                    #             speak("膝关节未伸直")
                    #         elif hip_angle > 55:
                    #             if hip_angle < 70:
                    #                 speak("正常")
                    #             if hip_angle > 70:
                    #                 speak("优秀")


            except Exception as e:
                print(f"姿势检测错误: {str(e)}")

            self.current_frame = annotated_frame

            # 转换视频帧格式
            rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            h, w, ch = rgb_image.shape
            bytes_per_line = ch * w
            qt_image = QImage(rgb_image.data, w, h, bytes_per_line, QImage.Format.Format_RGB888)
            self.change_pixmap.emit(qt_image)

        cap.release()

    def stop(self):
        self.running = False

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.init_ui()
        self.init_video_thread()

    def init_ui(self):
        self.setWindowTitle("智能姿势检测系统")
        self.setGeometry(100, 100, 1280, 720)
        self.setStyleSheet("""
            background: #2C3E50; 
            color: #ECF0F1;
            QLineEdit {
                padding: 12px;
                font-size: 16px;
                border: 2px solid #3498DB;
                border-radius: 5px;
            }
            QPushButton {
                background: #3498DB;
                padding: 12px;
                border-radius: 5px;
                font-weight: bold;
            }
            QPushButton:hover {
                background: #2980B9;
            }
        """)

        # 创建UI组件
        self.image_label = QLabel()
        self.image_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.id_input = QLineEdit(placeholderText="输入受试者编号")
        self.confirm_btn = QPushButton("确认编号", clicked=self.validate_id)
        self.capture_btn = QPushButton("立即拍照", clicked=self.manual_save)
        self.capture_btn.setStyleSheet("""
            QPushButton {
                background: #27AE60;
                padding: 12px;
                border-radius: 5px;
                font-weight: bold;
            }
            QPushButton:hover {
                background: #219A52;
            }
        """)
        

        # 布局设置
        top_layout = QHBoxLayout()
        top_layout.addWidget(self.id_input)
        top_layout.addWidget(self.confirm_btn)
        top_layout.addWidget(self.capture_btn)

        main_layout = QVBoxLayout()
        main_layout.addLayout(top_layout)
        main_layout.addWidget(self.image_label, stretch=1)
        
        container = QWidget()
        container.setLayout(main_layout)
        self.setCentralWidget(container)

    def manual_save(self):
        """手动保存按钮点击处理"""
        if not self.id_input.text().strip():
            QMessageBox.warning(self, "警告", "请先输入并确认受试者编号")
            return
        self.video_thread.capture_signal.emit()

    def init_video_thread(self):
        self.video_thread = VideoThread()
        self.video_thread.change_pixmap.connect(self.update_frame)
        self.video_thread.save_signal.connect(self.save_image)
        self.video_thread.speak_signal.connect(self.speak)
        self.video_thread.error_signal.connect(self.show_error)
        self.video_thread.start()

    def validate_id(self):
        if not self.id_input.text().strip():
            QMessageBox.warning(self, "输入错误", "受试者编号不能为空")
            return
        QMessageBox.information(self, "验证成功", "编号已确认，可以开始检测")

    @pyqtSlot(QImage)
    def update_frame(self, image):
        self.image_label.setPixmap(
            QPixmap.fromImage(image).scaled(
                1280, 720, 
                Qt.AspectRatioMode.KeepAspectRatio,
                Qt.TransformationMode.SmoothTransformation
            )
        )

    @pyqtSlot()
    def save_image(self):
        subject_id = self.id_input.text().strip()
        if not subject_id:
            self.speak("未识别到受试者编号")
            return

        timestamp = time.strftime("%Y%m%d_%H%M%S")
        save_type = "manual" if self.sender() == self.capture_btn else "auto"
        filename = f"{subject_id}_{save_type}_{timestamp}.png"
        cv2.imwrite(filename, self.video_thread.current_frame)
        self.speak(f"已保存姿势数据: {filename}")

    @pyqtSlot(str)
    def speak(self, message):
        engine.say(message)
        engine.runAndWait()

    @pyqtSlot(str)
    def show_error(self, message):
        QMessageBox.critical(self, "硬件错误", message)
        self.close()

    def closeEvent(self, event):
        self.video_thread.stop()
        self.video_thread.wait(3000)
        event.accept()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    app.setFont(QFont("Arial", 12))
    window = MainWindow()
    window.show()
    sys.exit(app.exec())

In [8]:
!pip install pyaudio

Looking in indexes: https://pypi.org/simple/
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp39-cp39-macosx_11_0_arm64.whl size=24181 sha256=f5430f85d1c577f4bb99e652fe77afeb0ef449a0898f95607415be2a6e065bb9
  Stored in directory: /Users/zehaokou/Library/Caches/pip/wheels/28/d3/62/6ad369dc09fe82e1c9ceb83601a800eb305b901df7789aa550
Successfully built pyaudio


In [1]:
import cv2
import mediapipe as mp
import math
import pyttsx3
import time
import os
import speech_recognition as sr  
from datetime import datetime
from docx import Document
from docx.shared import Inches

# 初始化MediaPipe和pyttsx3
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils
engine = pyttsx3.init()

# 语音播报
def speak(message):
    engine.say(message)
    engine.runAndWait()

# 语音识别
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("请回答：“是” 或 “否”")
        speak("请回答：“是” 或 “否”")
        recognizer.adjust_for_ambient_noise(source)  
        try:
            audio = recognizer.listen(source, timeout=5)  
            text = recognizer.recognize_google(audio, language="zh-CN")  
            print(f"识别结果: {text}")
            if "是" in text:
                return "是"
            elif "否" in text:
                return "否"
            else:
                speak("未听清，请再说一次")
                return recognize_speech()  
        except sr.UnknownValueError:
            speak("未听清，请再说一次")
            return recognize_speech()
        except sr.RequestError:
            speak("语音识别服务不可用，请稍后再试")
            return None

# 计算角度
def calculate_angle(a, b, c):
    ab = [b[0] - a[0], b[1] - a[1]]
    bc = [c[0] - b[0], c[1] - b[1]]
    dot_product = ab[0] * bc[0] + ab[1] * bc[1]
    cross_product = ab[0] * bc[1] - ab[1] * bc[0]
    angle = math.degrees(math.atan2(abs(cross_product), dot_product))
    return angle

# 生成 Word 报告
def generate_report_with_image(image_path, report_content, report_path):
    doc = Document()
    doc.add_heading('姿势检测报告', 0)
    doc.add_paragraph(f"报告生成时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    doc.add_paragraph(f"症状描述：{report_content}")
    doc.add_paragraph("照片：")
    doc.add_picture(image_path, width=Inches(3))
    doc.save(report_path)

# 获取桌面路径
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

# 初始化摄像头
cap = cv2.VideoCapture(0)
last_time = time.time()
hip_angle_history = []
leg_raised = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark

        left_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y)
        left_hip = (landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y)
        left_knee = (landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y)
        left_ankle = (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y)
        left_big_toe = (landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y)

        hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        ankle_angle = calculate_angle(left_knee, left_ankle, left_big_toe)

        hip_angle_complement = 180 - hip_angle
        knee_angle_complement = 180 - knee_angle
        ankle_angle_complement = 180 - ankle_angle

        # 在左上角显示角度信息
        cv2.putText(frame, f"Hip: {hip_angle_complement:.2f}°", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Knee: {knee_angle_complement:.2f}°", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f"Ankle: {ankle_angle_complement:.2f}°", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # if (100 < ankle_angle_complement < 110) and (173 < knee_angle_complement < 180) and (175 < hip_angle_complement < 180):
        if (100 < ankle_angle_complement < 180) and (160 < knee_angle_complement < 180) and (160 < hip_angle_complement < 180):
            speak("请抬左腿，保持膝盖打直和脚掌勾起，保持三秒")
            leg_raised = True

        if leg_raised:
            current_time = time.time()
            if current_time - last_time >= 1:
                last_time = current_time
                
                if knee_angle_complement < 170:
                    speak("膝关节未伸直")
                else:
                    hip_angle_history.append(hip_angle_complement)
                    if len(hip_angle_history) > 3:
                        hip_angle_history.pop(0)

                    if len(hip_angle_history) == 3 and abs(hip_angle_history[0] - hip_angle_history[2]) <= 5:
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        file_path = os.path.join(desktop_path, f"pose_{timestamp}.jpg")
                        cv2.imwrite(file_path, frame)

                        speak("请回答以下问题")
                        speak("是否牵扯")
                        answer_1 = recognize_speech()
                        if answer_1 == "是":
                            if hip_angle < 70:
                                report_content = "后侧链紧张"
                            report_content = "正常"
                        else:
                            speak("是否疼痛或不适")
                            answer_2 = recognize_speech()
                            if answer_2 == "是":
                                if hip_angle < 70:
                                    report_content = "骶髂关节僵紧"
                                report_content = "腰椎僵紧"
                            else:
                                speak("是否麻木")
                                answer_3 = recognize_speech()
                                if answer_3 == "是":
                                    report_content = "坐骨神经卡压"
                                else:
                                    report_content = "未出现明显症状"

                        report_path = os.path.join(desktop_path, f"report_{timestamp}.docx")
                        generate_report_with_image(file_path, report_content, report_path)

                        speak("报告已生成，并保存至桌面")
                        cap.release()
                        cv2.destroyAllWindows()
                        break

    cv2.imshow("Pose Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1742210097.457138   34412 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742210097.560256   34574 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742210097.573293   34574 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-03-17 19:14:58.424 python[2425:34412] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
W0000 00:00:1742210099.912759   34575 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
2025-03-17 19:15:04.466 py

KeyboardInterrupt: 

In [1]:
import sys
import cv2
import mediapipe as mp
import math
import pyttsx3
import time
import os
from datetime import datetime
from docx import Document
from docx.shared import Inches

from PyQt6.QtWidgets import (QApplication, QMainWindow, QLabel, QLineEdit, QPushButton,
                             QVBoxLayout, QWidget, QMessageBox)
from PyQt6.QtCore import QTimer, Qt
from PyQt6.QtGui import QImage, QPixmap

class PoseApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("姿势检测")
        self.subject_id = ""
        
        # UI控件：标签、输入框和按钮
        self.input_label = QLabel("请输入受试者编号:")
        self.subject_input = QLineEdit()
        self.start_button = QPushButton("开始检测")
        self.start_button.clicked.connect(self.start_detection)
        
        # 用于显示摄像头画面的标签
        self.video_label = QLabel()
        self.video_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        
        layout = QVBoxLayout()
        layout.addWidget(self.input_label)
        layout.addWidget(self.subject_input)
        layout.addWidget(self.start_button)
        layout.addWidget(self.video_label)
        
        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)
        
        # 初始化 mediapipe 和 pyttsx3
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose()
        self.mp_drawing = mp.solutions.drawing_utils
        self.engine = pyttsx3.init()
        
        # 摄像头和计时器
        self.cap = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        
        # 用于逻辑判断的变量
        self.last_time = time.time()
        self.hip_angle_history = []
        self.leg_raised = False

    def speak(self, message):
        # 语音播报并打印消息
        print(message)
        self.engine.say(message)
        self.engine.runAndWait()

    def calculate_angle(self, a, b, c):
        # 计算三点之间的夹角
        ab = [b[0] - a[0], b[1] - a[1]]
        bc = [c[0] - b[0], c[1] - b[1]]
        dot_product = ab[0] * bc[0] + ab[1] * bc[1]
        cross_product = ab[0] * bc[1] - ab[1] * bc[0]
        angle = math.degrees(math.atan2(abs(cross_product), dot_product))
        return angle

    def ask_question(self, question):
        # 使用消息对话框进行简单的“是/否”提问
        msg_box = QMessageBox(self)
        msg_box.setWindowTitle("提问")
        msg_box.setText(question + "\n点击 [是] 或 [否]")
        yes_button = msg_box.addButton("是", QMessageBox.ButtonRole.YesRole)
        no_button = msg_box.addButton("否", QMessageBox.ButtonRole.NoRole)
        msg_box.exec()
        if msg_box.clickedButton() == yes_button:
            return "是"
        else:
            return "否"

    def generate_report_with_image(self, image_path, report_content, report_path):
        doc = Document()
        doc.add_heading('姿势检测报告', 0)
        doc.add_paragraph(f"报告生成时间：{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        # 创建表格
        table = doc.add_table(rows=1, cols=2)
        table.style = 'Table Grid'
        hdr_cells = table.rows[0].cells
        hdr_cells[0].text = '症状'
        hdr_cells[1].text = '描述'
        for cell in hdr_cells:
            cell.width = Inches(3)
        symptoms = report_content.split('；')
        for symptom in symptoms:
            row_cells = table.add_row().cells
            symptom_parts = symptom.split('：')
            if len(symptom_parts) == 2:
                row_cells[0].text = symptom_parts[0]
                row_cells[1].text = symptom_parts[1]
        doc.add_paragraph("照片：")
        doc.add_picture(image_path, width=Inches(3))
        doc.save(report_path)

    def start_detection(self):
        # 获取受试者编号，并启动摄像头检测
        self.subject_id = self.subject_input.text().strip()
        if not self.subject_id:
            QMessageBox.warning(self, "警告", "请输入受试者编号!")
            return
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            QMessageBox.critical(self, "错误", "无法打开摄像头!")
            return
        self.timer.start(30)  # 每30毫秒更新一次画面
        self.start_button.setEnabled(False)
        self.subject_input.setEnabled(False)

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            return
        # 转为RGB并处理姿势检测
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(frame_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = (landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
            left_hip = (landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[self.mp_pose.PoseLandmark.LEFT_HIP.value].y)
            left_knee = (landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[self.mp_pose.PoseLandmark.LEFT_KNEE.value].y)
            left_ankle = (landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[self.mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
            left_big_toe = (landmarks[self.mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,
                            landmarks[self.mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y)
            hip_angle = self.calculate_angle(left_shoulder, left_hip, left_knee)
            knee_angle = self.calculate_angle(left_hip, left_knee, left_ankle)
            ankle_angle = self.calculate_angle(left_knee, left_ankle, left_big_toe)
            hip_angle_complement = 180 - hip_angle
            knee_angle_complement = 180 - knee_angle
            ankle_angle_complement = 180 - ankle_angle

            # 在画面上显示角度信息
            cv2.putText(frame, f"hip: {hip_angle_complement:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"knee: {knee_angle_complement:.2f}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"ankle: {ankle_angle_complement:.2f}", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # 满足条件时，提示受试者抬腿
            if (100 < ankle_angle_complement < 110) and (173 < knee_angle_complement < 180) and (175 < hip_angle_complement < 180):
                self.speak("请抬左腿，保持膝盖打直和脚掌勾起，抬至最高点保持三秒")
                self.leg_raised = True

            if self.leg_raised:
                if hip_angle_complement < 150:
                    current_time = time.time()
                    if current_time - self.last_time >= 1:
                        self.last_time = current_time
                        if knee_angle_complement < 170:
                            self.speak("膝关节未伸直")
                        else:
                            if len(self.hip_angle_history) > 0:
                                hip_angle_diff = abs(hip_angle_complement - self.hip_angle_history[-1])
                            else:
                                hip_angle_diff = float('inf')
                            self.hip_angle_history.append(hip_angle_complement)
                            if len(self.hip_angle_history) > 3:
                                self.hip_angle_history.pop(0)
                            if len(self.hip_angle_history) > 2 and hip_angle_diff < 5:
                                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                                file_name = f"pose_{self.subject_id}_{timestamp}.jpg"
                                file_path = os.path.join(os.path.expanduser("~"), "Desktop", file_name)
                                cv2.putText(frame, f"Lift angle: {180-self.hip_angle_history[2]:.2f}", (20, 160),
                                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                                cv2.imwrite(file_path, frame)
                                self.speak("照片已拍摄并保存")
                                
                                self.speak("请回答以下问题，若答案为是则点击[是]，否则点击[否]")
                                answer_1 = self.ask_question("是否麻木？")
                                if answer_1 == "是":
                                    report_content = (
                                        "神经：坐骨神经卡压、紧张；关节：髋关节囊僵紧；"
                                        "软组织：（梨状肌、股二头肌）水肿、卡压，血液循环差；"
                                        "代偿：骨盆、腰椎代偿；无力：核心、臀肌、腘绳肌无力；"
                                        "调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                    )
                                else:
                                    answer_2 = self.ask_question("是否疼痛或不适？")
                                    if answer_2 == "是":
                                        if hip_angle > 70:
                                            report_content = (
                                                "神经：腰骶丛神经紧张；关节：腰椎小关节僵紧；"
                                                "软组织：腰方肌、腰大肌、臀肌紧张，血液循环差；"
                                                "代偿：骨盆代偿；无力：核心无力；"
                                                "调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                            )
                                        else:
                                            report_content = (
                                                "神经：坐骨神经紧张；关节：骶髂关节、髋关节囊僵紧；"
                                                "软组织：后表链紧张，臀肌、腘绳肌、小腿三头肌紧张，血液循环差；"
                                                "代偿：腰椎代偿；无力：髂腰肌、后表链肌群紧张且无力；"
                                                "调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                            )
                                    else:
                                        answer_3 = self.ask_question("是否牵扯？")
                                        if answer_3 == "是":
                                            if hip_angle > 70:
                                                report_content = (
                                                    "神经：腰骶丛神经紧张；关节：髋关节囊僵紧；"
                                                    "软组织：腘绳肌紧张，血液循环差；代偿：骨盆代偿；"
                                                    "无力：核心无力；调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                                )
                                            else:
                                                report_content = (
                                                    "神经：坐骨神经紧张；关节：髋关节囊僵紧；"
                                                    "软组织：臀肌、腘绳肌、小腿三头肌紧张，血液循环差；"
                                                    "代偿：腰椎代偿；无力：髂腰肌、后表链肌群紧张且无力；"
                                                    "调整：松解、拉伸受限组织，训练肌肉离心能力，增强肌力、稳定性与协调性"
                                                )
                                        else:
                                            report_content = "未出现明显症状"
                                report_name = f"report_{self.subject_id}_{timestamp}.docx"
                                report_path = os.path.join(os.path.expanduser("~"), "Desktop", report_name)
                                self.generate_report_with_image(file_path, report_content, report_path)
                                self.speak(f"报告已生成并保存至桌面")
                                self.timer.stop()
                                self.cap.release()
                                cv2.destroyAllWindows()
        
        # 将 frame 转为 QImage 显示到 QLabel 上
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = frame_rgb.shape
        bytes_per_line = ch * w
        q_img = QImage(frame_rgb.data, w, h, bytes_per_line, QImage.Format.Format_RGB888)
        self.video_label.setPixmap(QPixmap.fromImage(q_img))
        
    def closeEvent(self, event):
        self.timer.stop()
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()
        event.accept()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = PoseApp()
    window.show()
    sys.exit(app.exec())

I0000 00:00:1742362175.785872 1833462 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1742362175.880742 2375489 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742362175.895567 2375494 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-03-19 13:29:37.432 python[48317:1833462] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-19 13:29:37.433 python[48317:1833462] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-03-19 13:29:39.748 python[48317:1833462] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
2025-03-19 13:29:43.239 python[48317:1833462] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use 

请抬左腿，保持膝盖打直和脚掌勾起，抬至最高点保持三秒
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
膝关节未伸直
照片已拍摄并保存
请回答以下问题，若答案为是则点击[是]，否则点击[否]
报告已生成并保存至桌面


SystemExit: 0

/Users/zehaokou/miniconda3/envs/rehab/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
